# **기본**

**필요한 라이브러리 호출**

**cuda를 사용하고 있는지, cpu를 사용하고 있는지 출력하기**

 **재현성을 유지할 수 있는 코드**

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import random

USE_CUDA=torch.cuda.is_available()
device=torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다: ",device)

random.seed(7777)
torch.manual_seed(7777)
if device=='cuda':
  torch.cuda.manual_seed_all(7777)



다음 기기로 학습합니다:  cuda


# **CNN모델 구축**

In [6]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layer1=nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2)) #nn.MaxPool2d(kernel_size=2,stride=2)
    self.layer2=nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2))
    self.fc=nn.Sequential(
        nn.Linear(7*7*64,100),
        nn.ReLU(),
        nn.Linear(100,10))

  def forward(self,x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=out.view(out.size(0),-1)
    out=self.fc(out)
    return out



# **mnist데이터 셋에서 train 데이터와 test 데이터 불러오기**

In [3]:
mnist_train=dsets.MNIST(root='MNIST_data/',train=True,transform=transforms.ToTensor(),download=True)
mnist_test=dsets.MNIST(root='MNIST_data/',train=False,transform=transforms.ToTensor(),download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 14.4MB/s]


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 482kB/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.38MB/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.51MB/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



# **하이퍼파라미터 설정과 DataLoader로 배치 설정**

In [4]:
training_epochs=3
batch_size=100

data_loader=DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True)

# **학습시작**

In [7]:
model=CNN().to(device) #모델을 gpu로 이동시켜야 한다. 합성곱은 많은 연산을 필요로하기때문에 gpu가 필요합니다.
criterion=nn.CrossEntropyLoss().cuda()  #손실함수도 cuda로 옮겨야 한다. #.cuda()대신, to(device)해도 됨.
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

for epoch in range(training_epochs):
  #이 학습에서 평균 손실 값을 계산할 것이다.
  avg_cost=0
  total_batch=len(data_loader)

  for idx,(x_train,y_train) in enumerate(data_loader):
    #데이터도 gpu로 옮겨줘야함.
    x_train=x_train.to(device)
    y_train=y_train.to(device)

    outputs=model(x_train)
    cost=criterion(outputs,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost+=cost/total_batch

  print('Epoch:','%04d'%(epoch+1),'cost=','{:.9f}'.format(avg_cost))

print('학습종료')

Epoch: 0001 cost= 0.173805118
Epoch: 0002 cost= 0.062940262
Epoch: 0003 cost= 0.055396762
학습종료


# **test데이터로 성능 체크**

In [11]:
with torch.no_grad(): #현재, 성능과 추론을 하는 단계이므로 gradient계산이 불필요하다. 따라서 torch.no_grad()를 이용하는 것임.
  x_test=mnist_test.data.view(len(mnist_test),1,28,28).float().to(device) #test의 데이터 갯수, channel 수, 높이, 너비 #역시 데이터를 gpu로 옮겨야 한다.
  y_test=mnist_test.targets.to(device)



#즉, 예측값이 맞으면 true, 틀리면 false로 데이터들을 매기고 .float()을 통해서 0,1로 바꾼 뒤, 평균을 내줘서 정확도를 계산한다.
  prediction=model(x_test)
  correct_prediction=torch.argmax(prediction,1)==y_test
  accuracy=correct_prediction.float().mean()
  print("정확도: ",accuracy.item())


정확도:  0.9788999557495117
